GENERACIÓN DE DATOS SINTÉTICOS PARA ANÁLISIS DE ENCUESTAS DE CALL CENTER PARA LA CALIBRACIÓN DE MODELOS

In [50]:
import random
import csv
import pandas as pd
from datetime import datetime, timedelta

# Canales de comunicación
canales = ["chatbot", "email", "phone"]

# Definir límites SLA por canal (en minutos)
sla_limits = {
    "chatbot": 10,    # 10 minutos
    "email": 1440,    # 24 horas
    "phone": 30       # 30 minutos
}

# Lista de grupos de resolución (hasta 10 grupos)
grupos = ["Grupo A", "Grupo B", "Grupo C", "Grupo D", "Grupo E", 
          "Grupo F", "Grupo G", "Grupo H", "Grupo I", "Grupo J"]

# Catálogo de clasificación de problemas
issue_classification_catalog = [
    "Card Issues", "Balance Inquiry", "Transaction Error", "Loan Inquiry", 
    "Payment Issues", "Account Setup", "Login Problems", "Fees Inquiry", 
    "Fraud Report", "General Inquiry"
]

# Ejemplos de comentarios en inglés para un call center bancario (100 por categoría)
comentarios = {
    "positive": [
        "The agent was very helpful and resolved my issue quickly.",
        "I’m impressed with how fast my loan application was processed.",
        "Great customer service, the phone representative was so polite!",
        "The chatbot answered all my questions perfectly, amazing!",
        "I received a detailed email response in less than an hour.",
        "The staff made transferring funds so easy and stress-free.",
        "Fantastic support over the phone, my issue was fixed in minutes.",
        "The chatbot helped me check my balance instantly, love it!",
        "The email support team was professional and courteous.",
        "I had a wonderful experience opening my account via phone.",
        "Quick and friendly service, my card issue was sorted out fast.",
        "The chatbot guided me through online banking setup flawlessly.",
        "The agent on the phone explained my mortgage options clearly.",
        "Super satisfied with the email response to my inquiry.",
        "The call center staff was patient and resolved my payment issue.",
        "The chatbot saved me so much time with my transaction query.",
        "I love how efficient the phone support was today!",
        "Email support was spot on, they fixed my login problem quickly.",
        "The agent was knowledgeable and made me feel valued.",
        "The chatbot gave me accurate info about my account in seconds.",
        "Phone support was excellent, they helped me with my statement.",
        "The email team responded promptly and solved my issue.",
        "The service was top-notch, I’m so happy with this bank!",
        "The chatbot made checking my interest rates so simple.",
        "The phone agent walked me through everything step-by-step.",
        "I got a fast and friendly response via email, great job!",
        "The support team over the phone was incredibly helpful.",
        "The chatbot was a lifesaver when I needed to block my card.",
        "Email support exceeded my expectations with their quick reply.",
        "The phone representative was kind and fixed my issue right away.",
        "The chatbot explained my fees so clearly, I’m impressed!",
        "The email team helped me update my contact info effortlessly.",
        "Phone support was amazing, they handled my complaint well.",
        "The chatbot was quick to assist with my transfer request.",
        "The email response was thorough and resolved my question.",
        "The phone agent was so patient with my account setup.",
        "The chatbot made paying my bill so easy and fast.",
        "Email support was polite and answered all my questions.",
        "The phone team resolved my overdraft issue professionally.",
        "The chatbot assisted me with my savings account inquiry perfectly.",
        "The email support team was efficient and friendly.",
        "Phone support made me feel confident about my banking.",
        "The chatbot quickly helped me reset my password.",
        "The email response was clear and solved my problem.",
        "The phone agent was great at explaining my loan terms.",
        "The chatbot handled my balance inquiry like a pro.",
        "Email support was fast and gave me all the info I needed.",
        "The phone team was courteous and fixed my issue instantly.",
        "The chatbot made banking so convenient, I love it!",
        "The email team helped me with my statement questions quickly.",
        "Phone support was outstanding, they deserve praise!",
        "The chatbot assisted me with my card replacement smoothly.",
        "The email response was professional and to the point.",
        "The phone agent helped me set up alerts with ease.",
        "The chatbot answered my mortgage question accurately.",
        "Email support was reliable and resolved my concern.",
        "The phone team made my experience so pleasant today.",
        "The chatbot was quick to assist with my payment query.",
        "The email team provided excellent service, I’m happy!",
        "Phone support was efficient and solved my problem fast.",
        "The chatbot helped me track my transaction effortlessly.",
        "The email response was detailed and very helpful.",
        "The phone agent was friendly and fixed my login issue.",
        "The chatbot made checking my rewards so simple.",
        "Email support was amazing, they responded right away.",
        "The phone team was professional and handled my request well.",
        "The chatbot guided me through my account options easily.",
        "The email team solved my issue with a single reply.",
        "Phone support was quick and made my day better!",
        "The chatbot assisted with my bill payment perfectly.",
        "The email response was prompt and very clear.",
        "The phone agent explained everything in a friendly way.",
        "The chatbot was fast and accurate with my inquiry.",
        "Email support made resolving my issue so easy.",
        "The phone team was great at helping me with my card.",
        "The chatbot answered my loan question in seconds.",
        "The email team was courteous and efficient.",
        "Phone support handled my transfer request flawlessly.",
        "The chatbot helped me with my account balance quickly.",
        "The email response was spot on and very professional.",
        "The phone agent was patient and solved my problem.",
        "The chatbot made resetting my PIN so simple.",
        "Email support was fast and fixed my login issue.",
        "The phone team was amazing with my inquiry.",
        "The chatbot assisted me with my fees effortlessly.",
        "The email team provided a clear and quick solution.",
        "Phone support was friendly and resolved my concern.",
        "The chatbot helped me with my savings goal easily.",
        "The email response was thorough and polite.",
        "The phone agent fixed my issue with a great attitude.",
        "The chatbot was quick to assist with my card issue.",
        "Email support was excellent and very responsive.",
        "The phone team made banking so easy for me.",
        "The chatbot handled my payment query perfectly.",
        "The email team was fast and solved my problem.",
        "Phone support was outstanding and very helpful.",
        "The chatbot assisted with my account setup smoothly.",
        "The email response was clear and professional.",
        "The phone agent helped me with my rewards program.",
        "The chatbot made my experience so convenient!"
    ],
    "negative": [
        "The phone agent was rude and didn’t solve my issue.",
        "The chatbot gave me wrong info about my account.",
        "Email support took days to reply, terrible service!",
        "I waited on hold for an hour, this is unacceptable.",
        "The chatbot couldn’t understand my simple question.",
        "The phone representative was unhelpful and impatient.",
        "Email support ignored my complaint completely.",
        "The service was slow, and my issue is still unresolved.",
        "The chatbot kept crashing when I needed help.",
        "The phone agent gave me conflicting information.",
        "Email support was useless, they didn’t even try.",
        "I had a horrible experience with the phone support.",
        "The chatbot failed to process my payment request.",
        "The email team took too long to fix my login issue.",
        "The phone agent hung up on me mid-conversation.",
        "The chatbot gave me irrelevant answers every time.",
        "Email support was rude and unprofessional.",
        "The phone service was awful, no one knew what to do.",
        "The chatbot couldn’t help with my card replacement.",
        "The email response was vague and unhelpful.",
        "I waited forever on the phone, and they didn’t help.",
        "The chatbot misunderstood my loan inquiry completely.",
        "Email support didn’t even read my message properly.",
        "The phone agent was clueless about my account.",
        "The chatbot crashed when I tried to check my balance.",
        "The email team ignored my urgent request.",
        "Phone support was a disaster, they made it worse!",
        "The chatbot gave me outdated info about my fees.",
        "The email response was late and full of errors.",
        "The phone agent refused to escalate my issue.",
        "The chatbot couldn’t handle my simple transfer request.",
        "Email support was slow and didn’t solve anything.",
        "The phone team was rude and hung up on me.",
        "The chatbot failed to reset my password correctly.",
        "The email team didn’t care about my problem.",
        "Phone support was terrible, they wasted my time.",
        "The chatbot gave me nonsense answers all the time.",
        "The email response was generic and useless.",
        "The phone agent didn’t listen to my complaint.",
        "The chatbot couldn’t assist with my bill payment.",
        "Email support took a week to reply, ridiculous!",
        "The phone team was unprofessional and unhelpful.",
        "The chatbot kept looping with no solution.",
        "The email team didn’t address my issue at all.",
        "Phone support was slow and didn’t fix my card issue.",
        "The chatbot failed to answer my mortgage question.",
        "The email response was rude and dismissive.",
        "The phone agent didn’t know basic banking info.",
        "The chatbot crashed during my transaction query.",
        "Email support was a complete letdown.",
        "The phone team made me wait and then hung up.",
        "The chatbot couldn’t process my balance request.",
        "The email team sent me a reply that made no sense.",
        "Phone support was awful, they ignored my problem.",
        "The chatbot gave me wrong loan terms info.",
        "The email response was delayed and unhelpful.",
        "The phone agent was impatient and rude.",
        "The chatbot failed to help with my account setup.",
        "Email support didn’t even try to fix my issue.",
        "The phone team was incompetent and slow.",
        "The chatbot couldn’t handle my payment issue.",
        "The email team took forever and didn’t help.",
        "Phone support was the worst I’ve ever experienced.",
        "The chatbot gave me incorrect balance info.",
        "The email response was sloppy and unclear.",
        "The phone agent didn’t care about my complaint.",
        "The chatbot failed to assist with my rewards query.",
        "Email support was slow and totally useless.",
        "The phone team didn’t solve my transfer problem.",
        "The chatbot kept freezing during my inquiry.",
        "The email team ignored my urgent card issue.",
        "Phone support was rude and disconnected me.",
        "The chatbot couldn’t reset my PIN properly.",
        "The email response was late and irrelevant.",
        "The phone agent was unhelpful with my fees.",
        "The chatbot failed to process my savings request.",
        "Email support didn’t bother to read my email.",
        "The phone team was slow and unprofessional.",
        "The chatbot gave me wrong payment instructions.",
        "The email team didn’t fix my login problem.",
        "Phone support was a nightmare, no help at all.",
        "The chatbot couldn’t assist with my statement.",
        "The email response was rude and incomplete.",
        "The phone agent didn’t understand my issue.",
        "The chatbot crashed when I asked about my loan.",
        "Email support was terrible, they ignored me.",
        "The phone team wasted my time with no solution.",
        "The chatbot gave me useless info about my card.",
        "The email team took days to reply with nothing useful.",
        "Phone support was disrespectful and unhelpful.",
        "The chatbot failed to help with my balance inquiry.",
        "The email response was vague and frustrating.",
        "The phone agent didn’t even try to assist me.",
        "The chatbot couldn’t handle my simple question.",
        "Email support was the worst, no effort at all.",
        "The phone team was slow and didn’t care.",
        "The chatbot crashed during my payment request.",
        "The email team sent a reply that solved nothing."
    ],
    "neutral": [
        "The phone support was okay, nothing special.",
        "The chatbot answered my question, but it took a while.",
        "Email support replied, but it wasn’t very detailed.",
        "The service was fine, I got what I needed.",
        "The phone agent helped me, but it was slow.",
        "The chatbot was average, did the job eventually.",
        "Email support was alright, nothing impressive.",
        "The phone team solved my issue, but it took time.",
        "The chatbot assisted me, but it wasn’t quick.",
        "The email response was decent, not great.",
        "Phone support was standard, no complaints.",
        "The chatbot handled my request, but it was basic.",
        "Email support was okay, could be better.",
        "The phone agent did their job, nothing more.",
        "The chatbot was fine, answered my balance query.",
        "The email team replied, but it was just okay.",
        "Phone support was average, got my issue fixed.",
        "The chatbot was alright, took a few tries.",
        "Email support was neutral, not bad or good.",
        "The phone team was okay, nothing standout.",
        "The chatbot helped with my payment, but it was slow.",
        "The email response was fair, not very thorough.",
        "Phone support was regular, solved my problem.",
        "The chatbot was decent, nothing exceptional.",
        "Email support was fine, met my expectations.",
        "The phone agent was alright, no big issues.",
        "The chatbot answered my loan query, but it was slow.",
        "The email team was okay, response was basic.",
        "Phone support was standard, nothing to note.",
        "The chatbot was average with my card request.",
        "Email support was neutral, did the minimum.",
        "The phone team was fine, no major problems.",
        "The chatbot helped me, but it wasn’t fast.",
        "The email response was okay, not very helpful.",
        "Phone support was decent, got the job done.",
        "The chatbot was alright with my transfer query.",
        "Email support was average, nothing special.",
        "The phone agent was okay, solved my issue slowly.",
        "The chatbot was fine, took time to assist.",
        "The email team was neutral, response was fair.",
        "Phone support was regular, nothing impressive.",
        "The chatbot handled my balance check, but it was slow.",
        "Email support was okay, not very detailed.",
        "The phone team was alright, no big deal.",
        "The chatbot was decent with my payment request.",
        "The email response was fine, not very great.",
        "Phone support was average, met basic needs.",
        "The chatbot was okay, assisted me eventually.",
        "Email support was neutral, not standout.",
        "The phone agent was fine, nothing remarkable.",
        "The chatbot answered my fees query, but it took time.",
        "The email team was okay, response was standard.",
        "Phone support was decent, no complaints really.",
        "The chatbot was alright with my account setup.",
        "Email support was fair, did what was needed.",
        "The phone team was neutral, solved my issue.",
        "The chatbot was fine with my loan question.",
        "The email response was average, not bad.",
        "Phone support was okay, nothing to rave about.",
        "The chatbot assisted with my card, but it was slow.",
        "Email support was decent, met expectations.",
        "The phone agent was alright, no issues.",
        "The chatbot was okay with my rewards query.",
        "The email team was neutral, response was fine.",
        "Phone support was standard, nothing exciting.",
        "The chatbot was average with my transfer request.",
        "Email support was okay, not very thorough.",
        "The phone team was fine, did their job.",
        "The chatbot helped with my bill, but it took time.",
        "The email response was decent, not great.",
        "Phone support was neutral, got it done.",
        "The chatbot was alright with my balance inquiry.",
        "Email support was fair, nothing special.",
        "The phone agent was okay, no big problems.",
        "The chatbot was fine with my payment query.",
        "The email team was neutral, response was basic.",
        "Phone support was average, nothing to note.",
        "The chatbot was decent with my card replacement.",
        "Email support was okay, not impressive.",
        "The phone team was alright, no complaints.",
        "The chatbot assisted with my fees, but it was slow.",
        "The email response was fine, not very helpful.",
        "Phone support was standard, met my needs.",
        "The chatbot was okay with my account question.",
        "Email support was neutral, did the minimum.",
        "The phone agent was fine, nothing standout.",
        "The chatbot was alright with my loan terms.",
        "The email team was decent, response was fair.",
        "Phone support was okay, nothing remarkable.",
        "The chatbot helped with my rewards, but it was slow.",
        "Email support was average, not bad or good.",
        "The phone team was neutral, solved my query.",
        "The chatbot was fine with my transfer issue.",
        "The email response was okay, not very detailed.",
        "Phone support was decent, no big issues.",
        "The chatbot was alright with my payment setup.",
        "Email support was fair, nothing exciting.",
        "The phone agent was okay, did the job."
    ]
}

# Función para generar una fecha aleatoria entre 2022 y 2025
def generar_fecha_aleatoria():
    inicio = datetime(2022, 1, 1)
    fin = datetime(2025, 2, 28)
    diferencia = (fin - inicio).days
    fecha_aleatoria = inicio + timedelta(days=random.randint(0, diferencia))
    return fecha_aleatoria

# Función para determinar el sentimiento basado en el mes (estacionalidad)
def get_seasonal_feeling(date):
    month = date.month
    if month in [7, 12]:  # Julio y Diciembre: más reclamos
        return random.choices(["positive", "negative", "neutral"], weights=[0.2, 0.7, 0.1])[0]
    else:  # Otros meses: proporciones normales
        return random.choices(["positive", "negative", "neutral"], weights=[0.6, 0.3, 0.1])[0]

# Función para generar sentiment_rate basado en el sentimiento
def generar_sentiment_rate(sentimiento):
    if sentimiento == "positive":
        return random.randint(4, 5)
    elif sentimiento == "negative":
        return random.randint(1, 2)
    else:  # neutral
        return 3

# Función para calcular el inicio de la semana (domingo)
def get_start_of_week(date):
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    days_to_sunday = (date_obj.weekday() + 1) % 7
    start_of_week = date_obj - timedelta(days=days_to_sunday)
    return start_of_week.strftime("%Y-%m-%d")

# Función para calcular el inicio del mes
def get_start_of_month(date):
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    start_of_month = date_obj.replace(day=1)
    return start_of_month.strftime("%Y-%m-%d")

# Función para generar un tiempo de respuesta ajustado al número de grupos
def generar_tiempo_respuesta(canal, sentimiento, num_grupos):
    # Tiempos base en minutos para cada canal y sentimiento
    base_time = {
        "chatbot": {"positive": (1, 5), "neutral": (5, 15), "negative": (10, 30)},
        "email": {"positive": (60, 240), "neutral": (240, 1440), "negative": (1440, 4320)},
        "phone": {"positive": (5, 15), "neutral": (15, 30), "negative": (30, 60)}
    }
    
    # Obtener el tiempo base según el canal y el sentimiento
    min_time, max_time = base_time[canal][sentimiento]
    
    # Determinar si el número de grupos afecta el tiempo de resolución
    if random.random() < 0.8:  # 80% de probabilidad de que los grupos afecten el tiempo
        # Ajustar el tiempo máximo con un multiplicador aleatorio y controlado
        multiplicador = 1 + (random.uniform(0.1, 0.3) * num_grupos)  # Ajuste fino: +10% a +30% por grupo
    else:
        # 20% de probabilidad de que los grupos no afecten el tiempo
        multiplicador = 1
    
    # Calcular el tiempo máximo ajustado
    adjusted_max_time = min(max_time * multiplicador, {
        "chatbot": 60,       # Límite superior para chatbot: 60 minutos
        "email": 14400,      # Límite superior para email: 14,400 minutos (10 días)
        "phone": 180         # Límite superior para phone: 180 minutos (3 horas)
    }[canal])
    
    # Generar un tiempo de respuesta aleatorio dentro del rango ajustado
    return random.randint(min_time, int(adjusted_max_time))

# Función para generar un CSAT rating según el sentimiento
def generar_csat(sentimiento):
    if sentimiento == "positive": return random.randint(4, 5)  # 4 o 5
    elif sentimiento == "negative": return random.randint(1, 2)  # 1 o 2
    else: return random.randint(3, 4)  # 3 o 4

# Función para determinar si está dentro del SLA (1 = sí, 0 = no)
def dentro_sla(canal, tiempo_respuesta):
    return 1 if tiempo_respuesta <= sla_limits[canal] else 0

# Función para determinar resolución al primer toque (1 = sí, 0 = no)
def generar_fcr(sentimiento):
    if sentimiento == "positive": return random.choices([1, 0], weights=[0.9, 0.1])[0]  # 90% probabilidad de 1
    elif sentimiento == "negative": return random.choices([1, 0], weights=[0.2, 0.8])[0]  # 20% probabilidad de 1
    else: return random.choices([1, 0], weights=[0.6, 0.4])[0]  # 60% probabilidad de 1

# Función para generar historial de grupos y conteos
def generar_group_history(sentimiento, canal):
    if sentimiento == "positive":
        num_grupos = random.choices([1, 2, 3], weights=[0.7, 0.2, 0.1])[0]  # Mayor probabilidad de pocos grupos
    elif sentimiento == "negative":
        num_grupos = random.choices([1, 2, 3, 4, 5], weights=[0.1, 0.2, 0.3, 0.3, 0.1])[0]  # Más grupos en negativos
    else:
        num_grupos = random.choices([1, 2, 3, 4], weights=[0.4, 0.3, 0.2, 0.1])[0]
    
    group_history = []
    for _ in range(num_grupos):
        group = random.choice(grupos)
        group_history.append(group)
    
    history_str = ",".join(group_history)
    unique_groups = len(set(group_history))  # Conteo de grupos únicos
    total_groups = len(group_history)  # Conteo total de grupos (con repeticiones)
    
    return history_str, unique_groups, total_groups

# Función para determinar el grupo que recibe la calificación CSAT (siempre el último)
def generar_csat_rated_group(group_history, fcr):
    groups = group_history.split(",")
    return groups[-1]  # Siempre el último grupo del historial

# Función para generar clasificación de problemas según sentimiento y canal
def generar_issue_classification(sentimiento, canal):
    weights = {
        "positive": [0.15, 0.20, 0.05, 0.20, 0.15, 0.10, 0.05, 0.05, 0.00, 0.05],  # Menos "Fraud Report"
        "neutral": [0.15, 0.20, 0.10, 0.15, 0.10, 0.10, 0.10, 0.05, 0.02, 0.03],
        "negative": [0.15, 0.05, 0.20, 0.05, 0.15, 0.05, 0.15, 0.05, 0.10, 0.05]   # Más "Transaction Error", "Fraud"
    }
    return random.choices(issue_classification_catalog, weights=weights[sentimiento])[0]

# Función para clasificar como PROMOTER o DETRACTOR según CSAT (en mayúsculas)
def get_classification(csat_rating):
    return "PROMOTER" if csat_rating >= 4 else "DETRACTOR"

# Generar datos sintéticos
datos = []
case_id = 15000  # ID inicial del caso
total_datos = 15000  # Total de entradas

for _ in range(total_datos):
    fecha = generar_fecha_aleatoria()  # Generar una fecha aleatoria
    sentimiento = get_seasonal_feeling(fecha)  # Determinar sentimiento según estacionalidad
    start_of_week = get_start_of_week(fecha.strftime("%Y-%m-%d"))  # Calcular inicio de la semana (domingo)
    start_of_month = get_start_of_month(fecha.strftime("%Y-%m-%d"))  # Calcular inicio del mes
    canal = random.choice(canales)  # Seleccionar un canal aleatorio
    sentiment_rate = generar_sentiment_rate(sentimiento)
    # Generar historial de grupos
    group_history, unique_groups, total_groups = generar_group_history(sentimiento, canal)
    
    # Generar tiempo de respuesta ajustado al número de grupos
    tiempo_respuesta = generar_tiempo_respuesta(canal, sentimiento, total_groups)
    
    csat = generar_csat(sentimiento)  # CSAT rating
    classification = get_classification(csat)  # Clasificación como PROMOTER o DETRACTOR
    within_sla = dentro_sla(canal, tiempo_respuesta)  # Dentro del SLA (1 o 0)
    fcr = generar_fcr(sentimiento)  # Resolución al primer toque (1 o 0)
    csat_rated_group = generar_csat_rated_group(group_history, fcr)  # Grupo que recibe CSAT (último del historial)
    issue_classification = generar_issue_classification(sentimiento, canal)  # Clasificación del problema
    
    datos.append({
        "case_id": f"CASE-{case_id}",
        "date": fecha.strftime("%Y-%m-%d"),
        "start_of_week": start_of_week,
        "start_of_month": start_of_month,
        "translated_comments": random.choice(comentarios[sentimiento]),
        "sentiment": sentimiento,
        "sentiment_rate": sentiment_rate, 
        "channel": canal,
        "resolution_time_min": tiempo_respuesta,
        "csat_rating_received": csat,
        "classification": classification,
        "resolved_in_sla": within_sla,
        "first_touch_resolution": fcr,
        "group_name_history": group_history,
        "groups": unique_groups,
        "total_groups": total_groups,
        "csat_rated_group_name": csat_rated_group,
        "issue_classification": issue_classification
    })
    case_id += 1

# Mezclar los datos para que no estén agrupados por sentimiento
random.shuffle(datos)



In [ ]:
# Crear dataframe
df = pd.DataFrame(datos)

# Ordenar para ver el valor máximo de RT
sorted_df = df.sort_values(by='resolution_time_min', ascending=False).head(5)

# Exportar DataFrame a CSV
df.to_csv('call_center_data.csv', index=False, sep=';', quotechar='"')
sorted_df

,case_id,date,start_of_week,start_of_month,translated_comments,sentiment,sentiment_rate,channel,resolution_time_min,csat_rating_received,classification,resolved_in_sla,first_touch_resolution,group_name_history,groups,total_groups,csat_rated_group_name,issue_classification
8686,CASE-20547,2023-09-10,2023-09-10,2023-09-01,The email team took too long to fix my login i...,negative,2,email,10411,2,DETRACTOR,0,1,"Grupo G,Grupo F,Grupo G,Grupo F,Grupo D",3,5,Grupo D,General Inquiry
12706,CASE-26878,2023-07-25,2023-07-23,2023-07-01,Email support didn’t even read my message prop...,negative,1,email,10361,2,DETRACTOR,0,0,"Grupo G,Grupo J,Grupo H,Grupo E,Grupo F",5,5,Grupo F,Fees Inquiry
10322,CASE-21924,2023-05-13,2023-05-07,2023-05-01,The phone agent didn’t care about my complaint.,negative,2,email,9897,2,DETRACTOR,0,0,"Grupo F,Grupo A,Grupo J,Grupo E,Grupo G",5,5,Grupo G,Login Problems
9252,CASE-29992,2023-06-09,2023-06-04,2023-06-01,The email response was vague and frustrating.,negative,1,email,9513,2,DETRACTOR,0,1,"Grupo G,Grupo D,Grupo A,Grupo J,Grupo I",5,5,Grupo I,Payment Issues
7368,CASE-21151,2024-12-20,2024-12-15,2024-12-01,The chatbot couldn’t help with my card replace...,negative,2,email,9360,2,DETRACTOR,0,1,"Grupo E,Grupo D,Grupo A,Grupo D,Grupo H",4,5,Grupo H,Fees Inquiry
